In [2]:
import numpy as np

In [3]:
tag2id, id2tag = {}, {}  # maps tag to id . tag2id: {"VB": 0, "NNP":1,..} , id2tag: {0: "VB", 1: "NNP"....}
word2id, id2word = {}, {} # maps word to id

for line in open('traindata.txt'):
    items = line.split('/')
    word, tag = items[0], items[1].rstrip()
    
    if word not in word2id:
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
        
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag
        
M, N = len(word2id), len(tag2id)   

In [4]:
pi = np.zeros(N) # the probability of a tag
A = np.zeros((N, M)) # given a tag, the probability of a word
B = np.zeros((N, N)) # transition matrix of tag

In [5]:
prev_tag = ''
for line in open('traindata.txt'):
    items = line.split('/')
    word, tag = items[0], items[1].rstrip()
    
    if prev_tag=='':
        pi[tag2id[tag]] += 1
        A[tag2id[tag]][word2id[word]] += 1
    else:
        B[tag2id[prev_tag]][tag2id[tag]] += 1
        A[tag2id[tag]][word2id[word]] += 1
        
    if word=='.':
        prev_tag = ''
    else:
        prev_tag = tag
        
pi /= sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])
        

In [6]:
def log(v):
    if v == 0:
        return np.log(v+0.000001)
    return np.log(v)

In [14]:
def viterbi(sent, A, B, pi):
    x = [word2id[x] for x in sent.split(' ')]
    T = len(x)
    
    dp = np.zeros((T, N))
    ptr = np.zeros((T,N), dtype=int)
    
    for i in range(N):
        dp[0][i] = log(pi[i]) + log(A[i][x[0]])
    
    for i in range(1,T):
        for j in range(N):
            dp[i][j] = -99999
            for k in range(N):
                score = dp[i-1][k] + log(A[j][x[i]]) + log(B[k][j])
                if score>dp[i][j]:
                    dp[i][j] = score
                    ptr[i][j] = k
                    
    # decoding: 把最好的tag sequence 打印出来
    best_seq = [0]*T  # best_seq = [1,5,2,23,4,...]  
    # step1: 找出对应于最后一个单词的词性
    best_seq[T-1] = np.argmax(dp[T-1])
    
    for i in range(T-2, -1, -1):
        best_seq[i] = ptr[i+1][best_seq[i+1]]
        
    for i in range(T):
        print(id2tag[best_seq[i]])

In [15]:
viterbi("I like playing piano", A, B, pi)

PRP
VBP
VBG
NN
